# Interpreting AMD's MIOpen Logs

I'm going to inspect operation logs obtained by running a [TensorFlow CNN benchmark script](https://github.com/tensorflow/benchmarks/tree/master/scripts/tf_cnn_benchmarks) on a [ROCm TensorFlow port](https://github.com/ROCmSoftwarePlatform/tensorflow-upstream) with [`MIOPEN_ENABLE_LOGGING=1`](https://github.com/ROCmSoftwarePlatform/MIOpen/wiki/MIOpen-Environment-Variables) set in the environment.

My goal is to extract a sequence of convolution, pooling, and batch normalization operations with corresponding parameters (tensor shapes) to verify the implementation correctness. The model I'm running is `resnet50`, with batch size set to `64` (the latter defines the `N` dimension).

## Basic parsing routines

The log file is read into memory and split by newline characters. Parsing routines transform the list of lines in-place: `next_operation` drops lines until encountering an operation header, `operation_name` extracts the operation name from the head of the line list and drops it, `operation_params` reads the operation parameters starting from the head of line list and advancing further until encountering the closing brace — you get the idea, once we go through the whole file, `len(lines)` should be `0`.

In [ ]:
def next_operation(lines):
  while not lines[0].startswith("MIOpen(HIP)"):
    del lines[0]
    if len(lines) == 0: return False
  return True


def operation_name(lines):
  header = lines.pop(0)
  op_name_args = header[28:] # skip 'MIOpen(HIP): miopenStatus_t '
  return op_name_args.split('(')[0]


def operation_params(lines):
  params = {}
  while True:
    param_line = lines.pop(0)
    if param_line == '}\n': break
    k, v = param_line.split(' = ')
    params[k] = v[:-1] # drop newline
  return params

## Decoding operation parameters

(Scarce) documentation is available for [convolution](https://github.com/ROCmSoftwarePlatform/MIOpen/wiki/MIOpen-Logger-to-Driver-Decoder-for-Convolutions), [pooling](https://github.com/ROCmSoftwarePlatform/MIOpen/wiki/MIOpen-Logger-to-Driver-Decoder-for-Pooling), and [batch normalization](https://github.com/ROCmSoftwarePlatform/MIOpen/wiki/MIOpen-Logger-to-Driver-Decoding-for-Batch-Normalization) operations.

In [ ]:
SUPPORTED_OPS = [
  'miopenConvolutionForward',
  'miopenBatchNormalizationForwardTraining',
  'miopenPoolingForward'
]

OP_NAMES = {
  'miopenConvolutionForward': 'Conv',
  'miopenBatchNormalizationForwardTraining': 'BatchNorm',
  'miopenPoolingMax': 'MaxPool'
}

def describe_operation(op_name, params):
  if op_name == 'miopenConvolutionForward':
    _, _, out_w, out_h = params['yDesc'].split(', ')
    
    out_chs, in_chs, kernel_w, kernel_h = params['wDesc'].split(', ')
    
    pad_h, pad_w, str_h, str_w, dil_h, dil_w, _ = params['convDesc'].split(', ')
    
    return [OP_NAMES[op_name], in_chs, out_chs, kernel_h, kernel_w, pad_h, str_h, out_h, out_w]
  elif op_name == 'miopenBatchNormalizationForwardTraining':
    _, in_chs, in_h, in_w = params['xDesc'].split(', ')
    
    return [OP_NAMES[op_name], in_chs, in_chs, '', '', '', '', in_h, in_w]
  elif op_name == 'miopenPoolingForward':
    _, in_chs, _, _ = params['xDesc'].split(', ')
    
    _, _, out_h, out_w = params['yDesc'].split(', ')
    
    pool_type, kernel_h, kernel_w, pad_h, pad_w, str_h, str_w, _ = params['poolDesc'].split(', ')
    
    return [OP_NAMES[pool_type], in_chs, in_chs, kernel_h, kernel_w, pad_h, str_h, out_h, out_w]

## Gluing everything together

In [ ]:
def extract_operation_sequence(log_path):
  operation_sequence = []
  
  with open(log_path) as f:
    lines = f.readlines()
  
  while next_operation(lines):
    op_name = operation_name(lines)
    if op_name in SUPPORTED_OPS:
      params = operation_params(lines)
      operation_sequence.append(
        describe_operation(op_name, params))

  return operation_sequence

In [ ]:
seq = extract_operation_sequence('./resnet50_b64_stdout_miopen_logging_no_warmup')
for d in seq[len(seq) - 108:-1]: print(','.join(d))